In [1]:
import numpy as np
import networkx as nx
import torch
from torch_geometric.data import Data
import time

import random
from torch_geometric.utils import from_networkx

import matplotlib.pyplot as plt

from torch_geometric.nn import SAGEConv

In [2]:
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

In [3]:
import torch.nn.functional as F

Step 1: Linearly transform node feature matrix
Step 2: Apply ReLU
Step 3: Linearly transform node feature matrix
Step 4: Apply mean aggregation
Step 5: Apply MLP

Step 4-5 can be easily done by either using the MP base class or graphsage.

In [4]:
import sys
sys.path.insert(1,'../../src')
from DataLoader import RGGDataset

In [5]:
DL = RGGDataset(root = '../../input')

In [6]:
dataset = DL.get(101)

In [7]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = torch.nn.Linear(1, 4)
        self.lin2 = torch.nn.Linear(4, 1)
        self.graphSage = SAGEConv(dataset.num_node_features, 1, root_weight=False, bias=False)

        #self.graphSage = SAGEConv(DL.get(101).num_node_features,1 ,root_weight = False, bias = False)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        #x = F.dropout(x, training=self.training)
        x = self.graphSage(x, edge_index)

        return x
        #F.log_softmax(x, dim=1)

In [8]:
model = GCN()

In [9]:
out = model(dataset)

In [22]:
torch.save(model.state_dict(), '../../models/GCNTwoLayersGraphSage')

In [10]:
torch.set_printoptions(precision=10) # Set displayed output precision to 10 digits

from torchquad import enable_cuda # Necessary to enable GPU support
from torchquad import Trapezoid, Simpson, Boole, MonteCarlo, VEGAS # The available integrators
import torchquad

13:30:42|TQ-INFO| Initializing torchquad.


In [11]:
eucl_norm = lambda x: torch.sqrt(torch.square(x[:,0]) + torch.square(x[:,1]))


def indicator_fct(x, distance, center):
    x = x
    center = center
    a = torch.add(x[:,0],-center[0])
    b = x[:,1]- center[1]
    c = torch.cat((a,b))
    d = torch.reshape(c,(2,len(x))).T
    z = (eucl_norm(d) < distance)*1
    return z

In [12]:
simp = Simpson() 

In [13]:
class cGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = torch.nn.Linear(1,4)
        self.lin2 = torch.nn.Linear(4,1)
        self.lin1.load_state_dict(model.lin1.state_dict())
        self.lin2.load_state_dict(model.lin2.state_dict())
        
        self.graphSage = SAGEConv(DL.get(101).num_node_features,1 ,root_weight = False, bias = False)
        self.graphSage.load_state_dict(model.graphSage.state_dict())
        
        self.weight = self.graphSage.lin_l.weight
        
    def forward(self, input_fct, radius = 0.2):
        self.input_fct = input_fct
        
        #def output_fct(x):
         #   return self.lin1(self.input_fct(x))
        
        def output_fct2(x):
            return F.relu(output_fct(x))
        
        #def output_fct3(x):
        #    return self.lin2(output_fct2(x))
        
        def output_fct(x):
            y = fct(x)
            b = torch.empty(( 0 ))
            for i, w in enumerate(y):
                b = torch.cat((b, self.lin1(torch.tensor([w]))), 0)
            d = torch.reshape(b,(len(x),4))
            return d

        def output_fct3(x):
            y = output_fct(x)
            b = torch.empty(( 0 ))
            for i, w in enumerate(y):
                b = torch.cat((b, self.lin2(torch.tensor(w))), 0)
            #d = torch.reshape(b,(4, len(x)))
            return b

        def output_fct4(position):
            g = lambda x, distance, center: indicator_fct(x, distance, position)*output_fct3(x)
            
            h = lambda x: g(x, radius, position)
            l = lambda x: indicator_fct(x, radius, position)
            
            int_of_f = simp.integrate(h,dim=2,N=1000,integration_domain = [[0,1],[0,1]])
            measure_Ball = simp.integrate(l,dim=2,N=1000,integration_domain = [[0,1],[0,1]])
        
            return torch.tensor(self.weight*int_of_f/measure_Ball)
        
        return output_fct4
    #F.log_softmax(x, dim=1)

In [14]:
cGCNN = cGCN()

In [21]:
torch.save(cGCNN.state_dict(), '../../cGCNTwoLayersGraphSage')

In [15]:
fct = lambda x: x[:,0]*x[:,1]


def main():
    #DL = RGGDataset(root = "../input", size = 1002)
    #model = torch.load("../models/graphSageOneLayer.pt")
    cGCNN = cGCN()
    #the_weight = (model.lin_l).weight
    #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #model = model.to(device)
    
    #cmodel = cGCNN.to(device)
    L2Errors = []
    for i in range(1, 202, 100):
        pos = DL.get(i).pos
        #pos = pos.to(device)
        cfct = cGCNN.forward(fct)
        
        b = torch.empty(( 0 ))
        for i, w in enumerate(pos):
            b = torch.cat((b, cfct(w)), 0)
        #d = torch.reshape(b,(4, len(x)))
        
        data = DL.get(i)
        #data = data.to(device)
        nodeErrors = b - model.forward(data.x, data.edge_index)
        L2Error = torch.sqrt(1/len(nodeErrors)*torch.sum(torch.pow(nodeErrors,2)))
        L2Errors.append(L2Error)
    return L2Errors

In [16]:
L2Errors = []
start = time.time()
for i in range(1, 202, 100):
    data = DL.get(i) 
    pos = data.pos
    #pos = pos.to(device)
    cfct = cGCNN.forward(fct)
        
    b = torch.empty(( 0 ))
    for i, w in enumerate(pos):
        b = torch.cat((b, cfct(w)), 0)
        #d = torch.reshape(b,(4, len(x)))
        
    #data = DL.get(i)
    #data = data.to(device)
    nodeErrors = b - model.forward(data)
    L2Error = torch.sqrt(1/len(nodeErrors)*torch.sum(torch.pow(nodeErrors,2)))
    L2Errors.append(L2Error)
end = time.time()
print(f"Took {(end-start)* 1000.0:.3f} ms")

13:30:43|TQ-INFO| Integration grid created.
<ipython-input-13-c895a07fad37>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.cat((b, self.lin2(torch.tensor(w))), 0)
13:30:43|TQ-INFO| Computed integral was tensor(-0.0490449592, grad_fn=<SumBackward1>).
13:30:43|TQ-INFO| Integration grid created.
13:30:43|TQ-INFO| Computed integral was tensor(0.0970370471).
<ipython-input-13-c895a07fad37>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.weight*int_of_f/measure_Ball)
13:30:43|TQ-INFO| Integration grid created.
13:30:43|TQ-INFO| Computed integral was tensor(-0.0629167929, grad_fn=<SumBackward1>).
13:30:43|TQ-INFO| Integration grid created.
13:30:43|TQ-INF

13:30:45|TQ-INFO| Computed integral was tensor(-0.0797047094, grad_fn=<SumBackward1>).
13:30:45|TQ-INFO| Integration grid created.
13:30:45|TQ-INFO| Computed integral was tensor(0.1160493940).
13:30:45|TQ-INFO| Integration grid created.
13:30:45|TQ-INFO| Computed integral was tensor(-0.0719534457, grad_fn=<SumBackward1>).
13:30:45|TQ-INFO| Integration grid created.
13:30:45|TQ-INFO| Computed integral was tensor(0.1269135922).
13:30:45|TQ-INFO| Integration grid created.
13:30:45|TQ-INFO| Computed integral was tensor(-0.0324157104, grad_fn=<SumBackward1>).
13:30:45|TQ-INFO| Integration grid created.
13:30:45|TQ-INFO| Computed integral was tensor(0.0620987713).
13:30:45|TQ-INFO| Integration grid created.
13:30:45|TQ-INFO| Computed integral was tensor(-0.0646383390, grad_fn=<SumBackward1>).
13:30:45|TQ-INFO| Integration grid created.
13:30:45|TQ-INFO| Computed integral was tensor(0.1249382868).
13:30:45|TQ-INFO| Integration grid created.
13:30:45|TQ-INFO| Computed integral was tensor(-0.05

13:30:47|TQ-INFO| Integration grid created.
13:30:48|TQ-INFO| Computed integral was tensor(-0.0702809915, grad_fn=<SumBackward1>).
13:30:48|TQ-INFO| Integration grid created.
13:30:48|TQ-INFO| Computed integral was tensor(0.1288889050).
13:30:48|TQ-INFO| Integration grid created.
13:30:48|TQ-INFO| Computed integral was tensor(-0.0703676865, grad_fn=<SumBackward1>).
13:30:48|TQ-INFO| Integration grid created.
13:30:48|TQ-INFO| Computed integral was tensor(0.1254321188).
13:30:48|TQ-INFO| Integration grid created.
13:30:48|TQ-INFO| Computed integral was tensor(-0.0422891527, grad_fn=<SumBackward1>).
13:30:48|TQ-INFO| Integration grid created.
13:30:48|TQ-INFO| Computed integral was tensor(0.0779012367).
13:30:48|TQ-INFO| Integration grid created.
13:30:48|TQ-INFO| Computed integral was tensor(-0.0353595801, grad_fn=<SumBackward1>).
13:30:48|TQ-INFO| Integration grid created.
13:30:48|TQ-INFO| Computed integral was tensor(0.0632098764).
13:30:48|TQ-INFO| Integration grid created.
13:30:48

13:30:50|TQ-INFO| Computed integral was tensor(0.0967901349).
13:30:50|TQ-INFO| Integration grid created.
13:30:50|TQ-INFO| Computed integral was tensor(-0.0739767551, grad_fn=<SumBackward1>).
13:30:50|TQ-INFO| Integration grid created.
13:30:50|TQ-INFO| Computed integral was tensor(0.1239506304).
13:30:50|TQ-INFO| Integration grid created.
13:30:50|TQ-INFO| Computed integral was tensor(-0.0631005242, grad_fn=<SumBackward1>).
13:30:50|TQ-INFO| Integration grid created.
13:30:50|TQ-INFO| Computed integral was tensor(0.0962963104).
13:30:50|TQ-INFO| Integration grid created.
13:30:50|TQ-INFO| Computed integral was tensor(-0.0343642980, grad_fn=<SumBackward1>).
13:30:50|TQ-INFO| Integration grid created.
13:30:50|TQ-INFO| Computed integral was tensor(0.0697530881).
13:30:50|TQ-INFO| Integration grid created.
13:30:50|TQ-INFO| Computed integral was tensor(-0.0476715043, grad_fn=<SumBackward1>).
13:30:50|TQ-INFO| Integration grid created.
13:30:50|TQ-INFO| Computed integral was tensor(0.090

13:30:53|TQ-INFO| Integration grid created.
13:30:53|TQ-INFO| Computed integral was tensor(0.1229629591).
13:30:53|TQ-INFO| Integration grid created.
13:30:53|TQ-INFO| Computed integral was tensor(-0.0461608097, grad_fn=<SumBackward1>).
13:30:53|TQ-INFO| Integration grid created.
13:30:53|TQ-INFO| Computed integral was tensor(0.0925925970).
13:30:53|TQ-INFO| Integration grid created.
13:30:53|TQ-INFO| Computed integral was tensor(-0.0699669644, grad_fn=<SumBackward1>).
13:30:53|TQ-INFO| Integration grid created.
13:30:53|TQ-INFO| Computed integral was tensor(0.1288889050).
13:30:53|TQ-INFO| Integration grid created.
13:30:53|TQ-INFO| Computed integral was tensor(-0.0424213745, grad_fn=<SumBackward1>).
13:30:53|TQ-INFO| Integration grid created.
13:30:53|TQ-INFO| Computed integral was tensor(0.0849382728).
13:30:53|TQ-INFO| Integration grid created.
13:30:53|TQ-INFO| Computed integral was tensor(-0.0689420179, grad_fn=<SumBackward1>).
13:30:53|TQ-INFO| Integration grid created.
13:30:53

13:30:55|TQ-INFO| Computed integral was tensor(-0.0721799284, grad_fn=<SumBackward1>).
13:30:55|TQ-INFO| Integration grid created.
13:30:55|TQ-INFO| Computed integral was tensor(0.1303703785).
13:30:55|TQ-INFO| Integration grid created.
13:30:55|TQ-INFO| Computed integral was tensor(-0.0650398359, grad_fn=<SumBackward1>).
13:30:55|TQ-INFO| Integration grid created.
13:30:55|TQ-INFO| Computed integral was tensor(0.1239506304).
13:30:55|TQ-INFO| Integration grid created.
13:30:55|TQ-INFO| Computed integral was tensor(-0.0707900301, grad_fn=<SumBackward1>).
13:30:55|TQ-INFO| Integration grid created.
13:30:55|TQ-INFO| Computed integral was tensor(0.0943209976).
13:30:55|TQ-INFO| Integration grid created.
13:30:55|TQ-INFO| Computed integral was tensor(-0.0700478405, grad_fn=<SumBackward1>).
13:30:55|TQ-INFO| Integration grid created.
13:30:55|TQ-INFO| Computed integral was tensor(0.1283950657).
13:30:55|TQ-INFO| Integration grid created.
13:30:55|TQ-INFO| Computed integral was tensor(-0.06

13:30:58|TQ-INFO| Integration grid created.
13:30:58|TQ-INFO| Computed integral was tensor(-0.0503313728, grad_fn=<SumBackward1>).
13:30:58|TQ-INFO| Integration grid created.
13:30:58|TQ-INFO| Computed integral was tensor(0.0995061770).
13:30:58|TQ-INFO| Integration grid created.
13:30:58|TQ-INFO| Computed integral was tensor(-0.0656584725, grad_fn=<SumBackward1>).
13:30:58|TQ-INFO| Integration grid created.
13:30:58|TQ-INFO| Computed integral was tensor(0.1288889050).
13:30:58|TQ-INFO| Integration grid created.
13:30:58|TQ-INFO| Computed integral was tensor(-0.0750827417, grad_fn=<SumBackward1>).
13:30:58|TQ-INFO| Integration grid created.
13:30:58|TQ-INFO| Computed integral was tensor(0.1244444549).
13:30:58|TQ-INFO| Integration grid created.
13:30:58|TQ-INFO| Computed integral was tensor(-0.0666067228, grad_fn=<SumBackward1>).
13:30:58|TQ-INFO| Integration grid created.
13:30:58|TQ-INFO| Computed integral was tensor(0.1244444400).
13:30:58|TQ-INFO| Integration grid created.
13:30:58

13:31:00|TQ-INFO| Computed integral was tensor(0.0450617336).
13:31:00|TQ-INFO| Integration grid created.
13:31:00|TQ-INFO| Computed integral was tensor(-0.0506909639, grad_fn=<SumBackward1>).
13:31:00|TQ-INFO| Integration grid created.
13:31:00|TQ-INFO| Computed integral was tensor(0.0967901349).
13:31:00|TQ-INFO| Integration grid created.
13:31:00|TQ-INFO| Computed integral was tensor(-0.0644401684, grad_fn=<SumBackward1>).
13:31:00|TQ-INFO| Integration grid created.
13:31:00|TQ-INFO| Computed integral was tensor(0.1202469245).
13:31:00|TQ-INFO| Integration grid created.
13:31:00|TQ-INFO| Computed integral was tensor(-0.0646036714, grad_fn=<SumBackward1>).
13:31:00|TQ-INFO| Integration grid created.
13:31:00|TQ-INFO| Computed integral was tensor(0.1274074167).
13:31:00|TQ-INFO| Integration grid created.
13:31:01|TQ-INFO| Computed integral was tensor(-0.0824833885, grad_fn=<SumBackward1>).
13:31:01|TQ-INFO| Integration grid created.
13:31:01|TQ-INFO| Computed integral was tensor(0.121

13:31:03|TQ-INFO| Integration grid created.
13:31:03|TQ-INFO| Computed integral was tensor(0.1288889050).
13:31:03|TQ-INFO| Integration grid created.
13:31:03|TQ-INFO| Computed integral was tensor(-0.0702438131, grad_fn=<SumBackward1>).
13:31:03|TQ-INFO| Integration grid created.
13:31:03|TQ-INFO| Computed integral was tensor(0.1234568059).
13:31:03|TQ-INFO| Integration grid created.
13:31:03|TQ-INFO| Computed integral was tensor(-0.0414291397, grad_fn=<SumBackward1>).
13:31:03|TQ-INFO| Integration grid created.
13:31:03|TQ-INFO| Computed integral was tensor(0.0524691418).
13:31:03|TQ-INFO| Integration grid created.
13:31:03|TQ-INFO| Computed integral was tensor(-0.0789908022, grad_fn=<SumBackward1>).
13:31:03|TQ-INFO| Integration grid created.
13:31:03|TQ-INFO| Computed integral was tensor(0.1234567985).
13:31:03|TQ-INFO| Integration grid created.
13:31:03|TQ-INFO| Computed integral was tensor(-0.0674735010, grad_fn=<SumBackward1>).
13:31:03|TQ-INFO| Integration grid created.
13:31:03

Took 22195.614 ms


In [17]:
L2Errors

[tensor(0.4382855594, grad_fn=<SqrtBackward>),
 tensor(0.0752069429, grad_fn=<SqrtBackward>),
 tensor(0.0750806779, grad_fn=<SqrtBackward>)]

from torchquad import enable_cuda

# Enable GPU support if available
enable_cuda() 

L2Errors = []

#model = torch.load("../models/graphSageOneLayer.pt")
#cGCNN = cGCN()
#the_weight = (model.graphSage.lin_l).weight
start = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
    
cGCNN = cGCNN.to(device)

for i in range(1, 202, 100):
    data = DL.get(i).to(device) 
    pos = data.pos
    #pos = pos.to(device)
    cfct = cGCNN.forward(fct)
        
    b = torch.empty(( 0 ))
    for i, w in enumerate(pos):
        b = torch.cat((b, cfct(w)), 0)
        #d = torch.reshape(b,(4, len(x)))
        
    #data = DL.get(i)
    #data = data.to(device)
    nodeErrors = b - model.forward(data)
    L2Error = torch.sqrt(1/len(nodeErrors)*torch.sum(torch.pow(nodeErrors,2)))
    L2Errors.append(L2Error)
end = time.time()
print(f"Took {(end-start)* 1000.0:.3f} ms")

L2Errors